In [ ]:
import re
import requests
import bs4
import json
import time


domain = 'https://www.lamoda.ru/'
urls_one = []
name_urls = []
link = []
all_link = []
product = []
all_reviews = []

start_url = 'https://www.lamoda.ru/brands/?b=beauty&sitelink=topmenuW&l=13'
s = requests.Session()
r = s.get(start_url)
assert r.ok
print(f'Page loading {start_url}')
soup = bs4.BeautifulSoup(r.text)
script_1 = str(soup.find_all('script')[15])
dic_one = script_1[script_1.find('payload: ') + 9: script_1.find(',settings: ')]
data_one = json.loads(dic_one)


for dt in data_one['data']:
    for i in dt['brands']:
        urls_one.append(i['url'])
        name_urls.append(i['name'])


for url, name in zip(urls_one, name_urls):    
    next_url_one = domain + url
    re = s.get(next_url_one)
    assert re.ok
    print(f'Downloading {next_url_one}')
    text = re.text
    href_soup = bs4.BeautifulSoup(text, 'lxml')

    paginator = href_soup.find_all('div', class_="paginator")
    if paginator:
        i = int(paginator[0]['data-page'])
        j = int(paginator[0]['data-pages'])
        while i <= j:
            print(f'Page {i}, brand - {name}')
            if i == 1:
                text = re.text
            else:    
                add_url = next_url_one + f'?page={i}'
                reg = s.get(add_url)
                assert reg.ok
                text = reg.text
            href_soup = bs4.BeautifulSoup(text, 'lxml')
            products = href_soup.select('a.products-list-item__link')
            for pr in products:
                if pr['href']:
                    link.append(pr['href'])
                    
                time.sleep(3)

                next_url_two = domain + pr['href']

                header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}

                re_two = requests.get(next_url_two, headers=header)
                assert re_two.ok
                print(f'Page loading {next_url_two}')
                soup_product = bs4.BeautifulSoup(re_two.text, 'lxml')
                script_2 = str(soup_product.find_all('script')[18])
                dic_two = script_2[script_2.find('payload: ') + 9: script_2.find(',settings: ')]
                data_two = json.loads(dic_two)

                product += [data_two['product']]
                all_reviews += [data_two['comments_data']]
                time.sleep(0.1)

                with open('product.json', 'w') as f:
                    f.write(json.dumps(product, indent=4, ensure_ascii=False))

                with open('all_reviews.json', 'w') as f:
                    f.write(json.dumps(all_reviews, indent=4, ensure_ascii=False))
                
            i = i + 1
    all_link.append(link)
    
    time.sleep(1)